<a href="https://colab.research.google.com/github/Muhammadsulton1/MIPT_Data_analys/blob/main/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B1%D1%80%D0%BE%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F_%D0%BE%D1%82%D0%B5%D0%BB%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
data = pd.read_csv('bookings.csv',sep=';')

In [ ]:
data.head()

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,stays total nights,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,2,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


In [ ]:
data.shape

(119390, 21)

In [ ]:
data.dtypes.value_counts()

object     10
int64      10
float64     1
dtype: int64

Предобработаем все наши колонки

In [ ]:
data.columns = map(str.lower, data.columns)

In [ ]:
data.columns = [x.replace(' ','_') for x in data.columns]

In [ ]:
data.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,stays_total_nights,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,2,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


Посмтрим с какой страны больше всего бронируют номера в отели и отменяют

In [ ]:
 country_reserv = data.groupby('country',as_index=False)\
  .agg({'reservation_status':'count','is_canceled':'sum'})\
  .sort_values('is_canceled',ascending=False)

In [ ]:
country_reserv

,country,reservation_status,is_canceled
135,PRT,48590,27519
59,GBR,12129,2453
51,ESP,8568,2177
56,FRA,10415,1934
81,ITA,3766,1333
...,...,...,...
44,DJI,1,0
143,SDN,1,0
40,CYM,1,0
39,CUB,8,0


In [ ]:
country_reserv['reservation_status'] - country_reserv['is_canceled']

135    21071
59      9676
51      6391
56      8481
81      2433
       ...  
44         1
143        1
40         1
39         8
0          2
Length: 177, dtype: int64

считаем среднее число остановки в отелях

In [ ]:
stays_night = data.groupby('hotel',as_index=False)\
  .agg({'stays_total_nights': 'mean'})

сравниваем на соответсвие был ли овербронинг

In [ ]:
data['compare'] = data['reserved_room_type'] == data['assigned_room_type']

In [ ]:
data['compare'].value_counts()

True     104473
False     14917
Name: compare, dtype: int64

In [ ]:
data.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,stays_total_nights,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,compare
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,True
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,True
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,False
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02,True
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,2,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,True


посмотрим бронирование отелей чаще всего на какой месяц приходилось

In [ ]:
S = data[data['arrival_date_year']== 2016] #2016 год

In [ ]:
A = data[data['arrival_date_year']== 2017] #2017 год

In [ ]:
A.groupby('arrival_date_month',as_index=False)\
  .agg({'arrival_date_year':'count'})\
  .sort_values('arrival_date_year',ascending=False)

,arrival_date_month,arrival_date_year
7,May,6313
0,April,5661
5,June,5647
4,July,5313
6,March,4970
1,August,4925
2,February,4177
3,January,3681


In [ ]:
S.groupby('arrival_date_month',as_index=False)\
  .agg({'arrival_date_year':'count'})\
  .sort_values('arrival_date_year',ascending=False)

,arrival_date_month,arrival_date_year
10,October,6203
8,May,5478
0,April,5428
11,September,5394
6,June,5292
1,August,5063
7,March,4824
5,July,4572
9,November,4454
3,February,3891


смотрим в каком году какого месяца чаще всего было отмена бронирования

In [ ]:
data.groupby(['arrival_date_year','arrival_date_month'],as_index=False)\
  .agg({'is_canceled': 'count'})\
  .sort_values(['arrival_date_year','is_canceled'],ascending=[False,False])\
  .groupby('arrival_date_year').head(1)

,arrival_date_year,arrival_date_month,is_canceled
25,2017,May,6313
16,2016,October,6203
5,2015,September,5114


In [ ]:
data.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,stays_total_nights,adults,children,babies
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119386.000000,119390.000000
mean,0.370416,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,3.427900,1.856403,0.103890,0.007949
std,0.482918,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,2.557439,0.579261,0.398561,0.097436
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,2.000000,0.000000,0.000000
50%,0.000000,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,3.000000,2.000000,0.000000,0.000000
75%,1.000000,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,4.000000,2.000000,0.000000,0.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,69.000000,55.000000,10.000000,10.000000


посчитаем наличие хотя бы одного ребенка

In [ ]:
data['total_kids'] = data['children'] + data['babies']

In [ ]:
data.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,stays_total_nights,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,compare,total_kids
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,True,0.0
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,True,0.0
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,False,0.0
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02,True,0.0
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,2,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,True,0.0


In [ ]:
data[data['hotel'] == 'Resort Hotel']['total_kids'].mean()

0.14258612081877184

In [ ]:
data[data['hotel'] == 'City Hotel']['total_kids'].mean()

0.09631142374505207

посчитаем у кого  меньше процент отмены бронирования в отели у кого есть дети(хотя бы один) или у кого нету

In [ ]:
data['has_kids'] = data['total_kids'] > 0

In [ ]:
no_kids_churn_rate = data.query('has_kids = False and is_canceled == 1').shape[0] / data.query('has_kids=False').shape[0]

In [ ]:
has_kids_churn_rate = data.query('has_kids = True and is_canceled == 1').shape[0] / data.query('has_kids=False').shape[0]